In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# Count viable channels for each montage

In [2]:
df = pd.read_parquet('/shared/rsaas/nschiou2/FOS/python/phase_data/pc/bandpass_only/phase_all_filt_chan.parquet')

In [3]:
df.replace(0, np.nan, inplace=True)

In [13]:
total_df = pd.DataFrame(columns=['subject', 'montage', 'n_trials', 'viable_channels'])

for subject in df.subject_id.unique():
    for montage in df.montage.unique():
        trial_result = dict()
        trial_result['subject'] = subject
        trial_result['montage'] = montage
        
        subset = df[(df['subject_id'] == subject) & (df['montage'] == montage)]
        subset = subset.loc[:, [c for c in subset.columns if c.startswith('ph')]]
        subset = subset.dropna(axis=1, how='all')
        trial_result['n_trials'] = subset.shape[0]
        trial_result['viable_channels'] = subset.shape[1] / 3
        
        total_df = total_df.append(trial_result, ignore_index=True)

In [20]:
total_df.sort_values(['subject', 'montage'], axis=0, inplace=True)
total_df.reset_index(drop=True, inplace=True)

In [ ]:
total_df

,subject,montage,n_trials,viable_channels
0,127,a,460,21.000000
1,127,b,460,32.000000
2,127,c,460,34.000000
3,127,d,460,35.000000
4,127,e,414,34.000000
5,127,f,460,34.000000
6,127,g,460,22.000000
7,127,h,460,24.000000
8,130,a,460,26.000000
9,130,b,460,37.000000


In [22]:
total_df.groupby('subject').sum()

,viable_channels
subject,
127,236.000000
130,230.000000
146,236.000000
149,178.000000
150,229.000000
151,213.000000
152,244.000000
153,234.000000
154,254.000000


In [23]:
total_df.groupby('montage').sum()

,viable_channels
montage,
a,444.000000
b,472.666667
c,446.000000
d,459.000000
e,435.000000
f,413.000000
g,403.000000
h,408.000000


# Impute missing channels with mean of other trials or random trial

In [2]:
import os
import random
from utils import constants

In [3]:
df = pd.read_parquet(os.path.join(constants.SUBJECTS_DIR, 'rs', 'bandpass_only', '154_e_4.parquet'))

In [4]:
df

,ph_e_0,ph_e_1,ph_e_2,ph_e_3,ph_e_4,ph_e_5,ph_e_6,ph_e_7,ph_e_8,ph_e_9,ph_e_10,ph_e_11,ph_e_12,ph_e_13,ph_e_14,ph_e_15,ph_e_16,ph_e_17,ph_e_18,ph_e_19,ph_e_20,ph_e_21,ph_e_22,ph_e_23,ph_e_24,ph_e_25,ph_e_26,ph_e_27,ph_e_28,ph_e_29,ph_e_30,ph_e_31,ph_e_32,ph_e_33,ph_e_34,ph_e_35,ph_e_36,ph_e_37,ph_e_38,ph_e_39,ph_e_40,ph_e_41,ph_e_42,ph_e_43,ph_e_44,ph_e_45,ph_e_46,ph_e_47,ph_e_48,ph_e_49,ph_e_50,ph_e_51,ph_e_52,ph_e_53,ph_e_54,ph_e_55,ph_e_56,ph_e_57,ph_e_58,ph_e_59,ph_e_60,ph_e_61,ph_e_62,ph_e_63,ph_e_64,ph_e_65,ph_e_66,ph_e_67,ph_e_68,ph_e_69,ph_e_70,ph_e_71,ph_e_72,ph_e_73,ph_e_74,ph_e_75,ph_e_76,ph_e_77,ph_e_78,ph_e_79,ph_e_80,ph_e_81,ph_e_82,ph_e_83,ph_e_84,ph_e_85,ph_e_86,ph_e_87,ph_e_88,ph_e_89,ph_e_90,ph_e_91,ph_e_92,ph_e_93,ph_e_94,ph_e_95,ph_e_96,ph_e_97,ph_e_98,ph_e_99,ph_e_100,ph_e_101,ph_e_102,ph_e_103,ph_e_104,ph_e_105,ph_e_106,ph_e_107,ph_e_108,ph_e_109,ph_e_110,ph_e_111,ph_e_112,ph_e_113,ph_e_114,ph_e_115,ph_e_116,ph_e_117,ph_e_118,ph_e_119,ph_e_120,ph_e_121,ph_e_122,ph_e_123,ph_e_124,ph_e_125,ph_e_126,ph_e_127,subject_id,trial_num,montage,freq_band,event,trial_type
0,0.0,2.943984,0.0,0.0,0.0,0.0,-3.914105,4.859165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.358373,6.591975,0.0,30.523751,0.0,0.0,4.066742,-10.826744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.474961,0.0,15.689806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.693600,5.481586,34.770003,-6.830109,-9.177944,0.0,0.0,-5.605722,0.0,15.434749,0.0,0.0,0.0,3.345367,0.0,-5.644573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.321021,2.149999,0.905103,0.0,0.0,0.0,17.101693,6.304169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.024939,13.261181,0.705408,23.008655,0.0,3.455086,23.006515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,6.628203,14.087211,0.0,0.0,154,0,e,7,320,5
1,0.0,2.526591,0.0,0.0,0.0,0.0,-8.206555,-6.842140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.102572,7.784291,0.0,37.182067,0.0,0.0,6.863667,3.316561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.624267,0.0,13.958289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9.089817,-3.715736,66.291549,22.722128,-1.683093,0.0,0.0,-8.862683,0.0,-5.806718,0.0,0.0,0.0,-5.059092,0.0,-18.029760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.134821,3.192345,-1.312587,0.0,0.0,0.0,2.594232,7.832148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.157833,4.077330,5.531055,51.552701,0.0,-1.390283,32.610673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,20.632006,-18.546841,0.0,0.0,154,0,e,7,476,5
2,0.0,-0.021436,0.0,0.0,0.0,0.0,-5.959891,-10.259101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.646535,2.856236,0.0,15.509799,0.0,0.0,7.496674,16.495612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.824383,0.0,-0.684277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.040179,-14.634870,62.342661,40.320428,11.864512,0.0,0.0,-7.303084,0.0,-22.124915,0.0,0.0,0.0,-10.322260,0.0,-17.907700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.519194,1.757965,-1.407627,0.0,0.0,0.0,-13.565557,3.008776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.354460,-6.120733,4.914739,40.154859,0.0,-10.199588,15.517074,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,10.294126,-32.379919,0.0,0.0,154,0,e,7,631,5
3,0.0,-2.393241,0.0,0.0,0.0,0.0,2.000116,-3.004553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.187121,-3.945187,0.0,-12.718570,0.0,0.0,5.803690,16.745248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.810578,0.0,-16.232089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.765132,-20.331395,19.444616,36.851449,17.670002,0.0,0.0,-1.012900,0.0,-20.530301,0.0,0.0,0.0,-10.124108,0.0,-5.437920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.170444,-1.001959,0.683024,0.0,0.0,0.0,-21.956514,-2.008890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.726160,-11.962863,2.039703,-1.619241,0.0,-14.057332,-16.406856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-10.499427,-20.014539,0.0,0.0,154,0,e,7,787,5
4,0.0,-2.807

In [5]:
df.loc[:, [c for c in df.columns if c.startswith('ph_')]] = df.loc[:, [c for c in df.columns if c.startswith('ph_')]].replace(0, np.nan)
df

,ph_e_0,ph_e_1,ph_e_2,ph_e_3,ph_e_4,ph_e_5,ph_e_6,ph_e_7,ph_e_8,ph_e_9,ph_e_10,ph_e_11,ph_e_12,ph_e_13,ph_e_14,ph_e_15,ph_e_16,ph_e_17,ph_e_18,ph_e_19,ph_e_20,ph_e_21,ph_e_22,ph_e_23,ph_e_24,ph_e_25,ph_e_26,ph_e_27,ph_e_28,ph_e_29,ph_e_30,ph_e_31,ph_e_32,ph_e_33,ph_e_34,ph_e_35,ph_e_36,ph_e_37,ph_e_38,ph_e_39,ph_e_40,ph_e_41,ph_e_42,ph_e_43,ph_e_44,ph_e_45,ph_e_46,ph_e_47,ph_e_48,ph_e_49,ph_e_50,ph_e_51,ph_e_52,ph_e_53,ph_e_54,ph_e_55,ph_e_56,ph_e_57,ph_e_58,ph_e_59,ph_e_60,ph_e_61,ph_e_62,ph_e_63,ph_e_64,ph_e_65,ph_e_66,ph_e_67,ph_e_68,ph_e_69,ph_e_70,ph_e_71,ph_e_72,ph_e_73,ph_e_74,ph_e_75,ph_e_76,ph_e_77,ph_e_78,ph_e_79,ph_e_80,ph_e_81,ph_e_82,ph_e_83,ph_e_84,ph_e_85,ph_e_86,ph_e_87,ph_e_88,ph_e_89,ph_e_90,ph_e_91,ph_e_92,ph_e_93,ph_e_94,ph_e_95,ph_e_96,ph_e_97,ph_e_98,ph_e_99,ph_e_100,ph_e_101,ph_e_102,ph_e_103,ph_e_104,ph_e_105,ph_e_106,ph_e_107,ph_e_108,ph_e_109,ph_e_110,ph_e_111,ph_e_112,ph_e_113,ph_e_114,ph_e_115,ph_e_116,ph_e_117,ph_e_118,ph_e_119,ph_e_120,ph_e_121,ph_e_122,ph_e_123,ph_e_124,ph_e_125,ph_e_126,ph_e_127,subject_id,trial_num,montage,freq_band,event,trial_type
0,NaN,2.943984,NaN,NaN,NaN,NaN,-3.914105,4.859165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.358373,6.591975,NaN,30.523751,NaN,NaN,4.066742,-10.826744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.474961,NaN,15.689806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.693600,5.481586,34.770003,-6.830109,-9.177944,NaN,NaN,-5.605722,NaN,15.434749,NaN,NaN,NaN,3.345367,NaN,-5.644573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.321021,2.149999,0.905103,NaN,NaN,NaN,17.101693,6.304169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.024939,13.261181,0.705408,23.008655,NaN,3.455086,23.006515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.628203,14.087211,NaN,NaN,154,0,e,7,320,5
1,NaN,2.526591,NaN,NaN,NaN,NaN,-8.206555,-6.842140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.102572,7.784291,NaN,37.182067,NaN,NaN,6.863667,3.316561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.624267,NaN,13.958289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.089817,-3.715736,66.291549,22.722128,-1.683093,NaN,NaN,-8.862683,NaN,-5.806718,NaN,NaN,NaN,-5.059092,NaN,-18.029760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.134821,3.192345,-1.312587,NaN,NaN,NaN,2.594232,7.832148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.157833,4.077330,5.531055,51.552701,NaN,-1.390283,32.610673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.632006,-18.546841,NaN,NaN,154,0,e,7,476,5
2,NaN,-0.021436,NaN,NaN,NaN,NaN,-5.959891,-10.259101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.646535,2.856236,NaN,15.509799,NaN,NaN,7.496674,16.495612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.824383,NaN,-0.684277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.040179,-14.634870,62.342661,40.320428,11.864512,NaN,NaN,-7.303084,NaN,-22.124915,NaN,NaN,NaN,-10.322260,NaN,-17.907700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.519194,1.757965,-1.407627,NaN,NaN,NaN,-13.565557,3.008776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.354460,-6.120733,4.914739,40.154859,NaN,-10.199588,15.517074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.294126,-32.379919,NaN,NaN,154,0,e,7,631,5
3,NaN,-2.393241,NaN,NaN,NaN,NaN,2.000116,-3.004553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.187121,-3.945187,NaN,-12.718570,NaN,NaN,5.803690,16.745248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.810578,NaN,-16.232089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.765132,-20.331395,19.444616,36.851449,17.670002,NaN,NaN,-1.012900,NaN,-20.530301,NaN,NaN,NaN,-10.124108,NaN,-5.437920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.170444,-1.001959,0.683024,NaN,NaN,NaN,-21.956514,-2.008890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.726160,-11.962863,2.039703,-1.619241,NaN,-14.057332,-16.406856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10.499427,-20.014539,NaN,NaN,154,0,e,7,787,5
4,NaN,-2.807817,NaN,NaN,NaN,NaN,

In [6]:
df = df.dropna(axis=1, how='all')
df

,ph_e_1,ph_e_6,ph_e_7,ph_e_20,ph_e_21,ph_e_23,ph_e_26,ph_e_27,ph_e_40,ph_e_42,ph_e_58,ph_e_59,ph_e_60,ph_e_61,ph_e_62,ph_e_65,ph_e_67,ph_e_71,ph_e_73,ph_e_84,ph_e_85,ph_e_86,ph_e_90,ph_e_91,ph_e_102,ph_e_103,ph_e_104,ph_e_105,ph_e_107,ph_e_108,ph_e_123,ph_e_124,ph_e_125,subject_id,trial_num,montage,freq_band,event,trial_type
0,2.943984,-3.914105,4.859165,7.358373,6.591975,30.523751,4.066742,-10.826744,2.474961,15.689806,-7.693600,5.481586,34.770003,-6.830109,-9.177944,-5.605722,15.434749,3.345367,-5.644573,0.321021,2.149999,0.905103,17.101693,6.304169,4.024939,13.261181,0.705408,23.008655,3.455086,23.006515,NaN,6.628203,14.087211,154,0,e,7,320,5
1,2.526591,-8.206555,-6.842140,12.102572,7.784291,37.182067,6.863667,3.316561,4.624267,13.958289,-9.089817,-3.715736,66.291549,22.722128,-1.683093,-8.862683,-5.806718,-5.059092,-18.029760,2.134821,3.192345,-1.312587,2.594232,7.832148,-2.157833,4.077330,5.531055,51.552701,-1.390283,32.610673,NaN,20.632006,-18.546841,154,0,e,7,476,5
2,-0.021436,-5.959891,-10.259101,8.646535,2.856236,15.509799,7.496674,16.495612,3.824383,-0.684277,-5.040179,-14.634870,62.342661,40.320428,11.864512,-7.303084,-22.124915,-10.322260,-17.907700,1.519194,1.757965,-1.407627,-13.565557,3.008776,-7.354460,-6.120733,4.914739,40.154859,-10.199588,15.517074,NaN,10.294126,-32.379919,154,0,e,7,631,5
3,-2.393241,2.000116,-3.004553,-0.187121,-3.945187,-12.718570,5.803690,16.745248,0.810578,-16.232089,-5.765132,-20.331395,19.444616,36.851449,17.670002,-1.012900,-20.530301,-10.124108,-5.437920,-1.170444,-1.001959,0.683024,-21.956514,-2.008890,0.726160,-11.962863,2.039703,-1.619241,-14.057332,-16.406856,NaN,-10.499427,-20.014539,154,0,e,7,787,5
4,-2.807817,9.751895,6.534871,-7.868654,-7.012484,-24.252539,0.417770,2.780461,-2.666596,-18.240280,-11.891889,-14.171308,-39.159762,15.177196,7.224825,6.916638,-3.432658,-4.611661,10.305452,-3.239473,-2.964897,2.392510,-17.551411,-2.846419,17.181384,-12.019021,1.008900,-39.376795,-6.998565,-37.485616,NaN,-20.123092,3.329420,154,0,e,7,943,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69727,-4.088063,2.848430,20.971964,-8.718757,-10.690595,-32.824392,-10.392190,-3.170067,11.586937,4.594457,-14.903962,23.322832,-2.775003,31.078384,-3.754330,1.086557,21.445066,-5.251651,-16.367429,-6.926569,-1.402275,-2.382093,-18.203063,4.760158,-10.572997,2.195350,-20.628436,3.425314,-45.753357,8.488237,22.145509,10.212775,39.151114,154,446,e,7,2969,12
69728,-5.665805,2.154226,20.325872,4.389575,-1.104380,-13.617330,0.346261,-7.187661,10.846940,5.735338,-12.669961,17.581433,-27.913200,41.984064,-2.900016,-0.765816,10.318298,1.775874,-33.401980,-3.057679,3.326676,-2.735077,-6.572988,2.579503,-32.667147,23.016442,-11.373817,4.537646,-26.181040,3.369988,17.906621,-13.310187,16.511917,154,446,e,7,3125,12
69729,-3.477541,0.772647,7.188789,8.074187,9.184442,13.219096,6.848514,-6.659511,2.505426,3.832626,-1.880247,-4.557029,-26.995805,20.984944,-0.802283,-0.160833,-16.655961,9.896062,-25.842226,3.389532,5.736639,-1.116297,5.900276,-3.812778,-37.529582,29.383354,5.330930,8.497530,15.536344,-6.875030,1.477050,-34.086710,-15.097092,154,446,e,7,3437,12
69730,1.373337,0.253010,-7.488665,3.110019,11.891939,25.025991,3.159880,1.426411,-6.084937,-0.390748,12.094530,-27.362761,-0.385402,-9.316059,0.726858,0.714660,-38.926222,11.491988,-2.100293,8.909946,3.651985,1.173512,10.050598,-9.133367,-22.009712,17.148343,17.147579,8.738379,44.569693,-10.880142,-6.518737,-28.423970,-34.171246,154,446,e,7,3593,12


In [7]:
na_cols = list(df.columns[df.isna().any()])
na_cols

['ph_e_123']

In [8]:
imputed_df = df.copy()

for col in na_cols:
    subset = df.loc[:, [col, 'trial_num']]

#     running_sig = list()
#     for trial in subset.trial_num.unique():
#         subset_subset = subset.loc[subset['trial_num'] == trial, :]
#         running_sig.append(subset_subset.loc[:, col].values)
#     running_sig = np.vstack(running_sig).T
#     mean_sig = np.nanmean(running_sig, axis=1)

#     for trial in subset.trial_num.unique():
#         subset_subset = subset.loc[subset['trial_num'] == trial, :]
#         if np.isnan(subset[col].values[0]):
#             idxs_to_replace = subset_subset.index
#             imputed_df.loc[idxs_to_replace, col] = mean_sig

    non_nan_df = df.dropna(axis=0, how='any', subset=[col])
    
    non_nan_trials = non_nan_df.trial_num.unique()
    non_nan_idx = np.zeros(len(subset.trial_num.unique()))
    for t in non_nan_trials:
        non_nan_idx = np.logical_or(non_nan_idx, subset.trial_num.unique() == t)
    assert len(non_nan_trials) == np.count_nonzero(non_nan_idx)
    non_nan_idx = np.argwhere(non_nan_idx).flatten()

    rand = random.choice(non_nan_idx)
    subset_subset = subset.loc[subset['trial_num'] == subset.trial_num.unique()[rand], :]
    rand_sig = subset_subset.loc[:, col].values
    assert not np.isnan(rand_sig[0])
    
    for trial in subset.trial_num.unique():
        subset_subset = subset.loc[subset['trial_num'] == trial, :]
        if np.isnan(subset[col].values[0]):
            idxs_to_replace = subset_subset.index
            imputed_df.loc[idxs_to_replace, col] = rand_sig

In [9]:
list(imputed_df.columns[imputed_df.isna().any()])

[]